In [1]:
%%time
import pandas as pd
df = pd.read_csv('HMAHCC_COMP.csv')

<string>:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 16.8 s


### Summary 
1. we have 14000 members
2. For now, I am only able to generalize a rule for data between day 0 to 180
3. Based on the current rule (see blocks below), [37.0]% of members have LTOT
4. There are 240 members have weird records, i.e. no PAY_DAY_SUPPLY_CNT data

### Next step
1. Do some more sanity checks on my current rule
2. Extend the rule to the data after day 180, consider situations that more than 1 Naive Opioid occured for a member

# I think that this is how we check for LTOT:

In [48]:
example = df[(df['PAY_DAY_SUPPLY_CNT'].notnull())  & (df['id']=='ID10024447278')][['Days', 'PAY_DAY_SUPPLY_CNT']].drop_duplicates()
example

,Days,PAY_DAY_SUPPLY_CNT
505,-614,5.0
512,-609,5.0
517,-575,5.0
612,0,3.0
620,4,30.0
628,32,30.0
637,66,30.0
643,94,30.0
658,122,30.0
663,150,30.0


In [81]:
example['run out on day'] = (example['Days'] + example['PAY_DAY_SUPPLY_CNT'])
# leave only Days >= 0 and <= 180 for to simplify the problem...for now. 
example = example[(example['Days']>=0) & (example['Days']<=180)]
example['cumulative use days'] = example['PAY_DAY_SUPPLY_CNT'].cumsum().shift(1)
example

,Days,PAY_DAY_SUPPLY_CNT,run out on day,cumulative use days
612,0,3.0,3.0,NaN
620,4,30.0,34.0,3.0
628,32,30.0,62.0,33.0
637,66,30.0,96.0,63.0
643,94,30.0,124.0,93.0
658,122,30.0,152.0,123.0
663,150,30.0,180.0,153.0
670,179,30.0,209.0,183.0


In [79]:
days_used_last_cycle =  min(180 - example['cumulative use days'].iloc[-1], example['PAY_DAY_SUPPLY_CNT'].iloc[-1])
days_used = example['cumulative use days'].iloc[-1] + days_used_last_cycle

days_used

180.0

### Apply to whole dataset

In [ ]:
import time
sec2 = df.groupby(by='id')

In [26]:
name = 'ID10013863216'
group = sec2.get_group(name)


def get_LTOT(group):
    if group[(group['Days']==0) & (group['PAY_DAY_SUPPLY_CNT'].notnull())].shape[0] == 0:
        return('No data')
    else:
        example = group[group['PAY_DAY_SUPPLY_CNT'].notnull()][['Days', 'PAY_DAY_SUPPLY_CNT']].drop_duplicates()
        example = example[(example['Days']>=0) & (example['Days']<=180)]
        example['cumulative use days'] = example['PAY_DAY_SUPPLY_CNT'].cumsum().shift(1)
        days_used_last_cycle =  min(180 - example['cumulative use days'].iloc[-1], example['PAY_DAY_SUPPLY_CNT'].iloc[-1])
        days_used = example['cumulative use days'].iloc[-1] + days_used_last_cycle

        if days_used >= 162:
            return(True)
        else:
            return(False)
        
print(get_LTOT(group))

True


In [17]:
%%time
sec = df.groupby(by='id')[['Days', 'PAY_DAY_SUPPLY_CNT']].agg(get_LTOT)

Wall time: 4min 43s


In [27]:
sec['Days'].value_counts()

False      8575
True       5185
No data     240
Name: Days, dtype: int64

In [25]:
sec['Days'].value_counts()[True]/sum(sec['Days'].value_counts())

0.3703571428571429

In [28]:
sec.to_csv('LTOTs_check_matt1.csv')